# Human-in-the-Loop for Agentic AI: HR Assistant Example

This notebook demonstrates the implementation of a Human-in-the-Loop (HITL) approach in an agentic AI system, specifically an HR assistant that helps employees manage their time off requests.

## 1. Introduction to Human-in-the-Loop (HITL) in Agentic AI

Human-in-the-Loop (HITL) is a paradigm where human judgment is incorporated into automated systems to improve decision-making, ensure safety, and enhance user experience. In the context of agentic AI, HITL refers to the collaboration between AI agents and humans, where humans provide oversight, confirmation, or additional input during the agent's operation.

HITL is particularly important in scenarios where:
- Decisions have significant consequences
- Complete automation might miss important context
- User confirmation adds a layer of safety and control
- The system needs to learn from human feedback

## 2. Core Concepts of Human-in-the-Loop

1. **Confirmation Requests** <br>
One of the fundamental HITL patterns is the confirmation request, where the AI agent asks for human approval before executing a potentially impactful action. This creates a checkpoint where humans can review and either confirm or reject the proposed action.

2. **Progressive Disclosure** <br>
HITL systems often implement progressive disclosure, where information and options are presented gradually to avoid overwhelming the user. The agent guides the human through a decision-making process step by step.

3. **Feedback Loops** <br>
Effective HITL systems incorporate feedback loops where human input is used to improve the agent's future performance. This can range from explicit feedback mechanisms to implicit learning from user interactions.

4. **Transparency** <br>
HITL requires transparency in the agent's reasoning and actions. Users need to understand what the agent is doing and why, especially when they're asked to make decisions based on the agent's recommendations.

5. **Graceful Handoffs** <br>
Well-designed HITL systems handle transitions between automated and human control smoothly. This includes clear signaling when human input is needed and providing sufficient context for humans to make informed decisions.

## 3. Implementation Example: HR Assistant Agent

The following code implements an HR assistant agent that helps employees manage their time off requests. This example demonstrates several HITL concepts, particularly the confirmation request pattern where the agent seeks explicit user approval before finalizing time off requests.

<img src="images/hr-assistant-architecture.png" width="800" height="400">

### 3.1 Setup and Imports

First, we import the necessary libraries and set up our environment:

In [ ]:
import os
import json
from datetime import datetime, timedelta
from strands import Agent, tool
from strands.models import BedrockModel
from aws_config import get_bedrock_session

### 3.2 Mock Database

For demonstration purposes, we'll use a simple dictionary to simulate a database of employee time off information:

In [ ]:
# Mock database for time off information
employee_time_off = {
    "total_days": 25,
    "used_days": 10,
    "pending_requests": []
}

### 3.3 Tool Functions

The agent uses tool functions to interact with the time off system. 

#### 3.3.1 Information Retrieval Tool

The `get_time_off()` tool retrieves information without requiring human confirmation, as it's a read-only operation with no significant consequences:

In [ ]:
@tool
def get_time_off() -> str:
    """Get the current time off balance for the employee.
    
    Returns:
        A JSON string containing the employee's time off information.
    """
    remaining_days = employee_time_off["total_days"] - employee_time_off["used_days"]
    
    response = {
        "total_days": employee_time_off["total_days"],
        "used_days": employee_time_off["used_days"],
        "remaining_days": remaining_days,
        "pending_requests": employee_time_off["pending_requests"]
    }
    
    return json.dumps(response)

#### 3.3.2 HITL Confirmation Request Pattern

The `request_time_off()` tool demonstrates the HITL confirmation request pattern. Instead of immediately submitting the time off request, it returns a confirmation request that requires explicit human approval:

In [ ]:
@tool
def request_time_off(start_date: str, number_of_days: int) -> str:
    """Submit a time off request for the employee.
    
    Args:
        start_date: The date that time off starts (format: YYYY-MM-DD)
        number_of_days: The number of days user wants to request off
    
    Returns:
        A JSON string containing the result of the time off request or a confirmation request.
    """
    try:
        # Parse the start date
        start_date_obj = datetime.strptime(start_date, "%Y-%m-%d")
        
        # Check if the date is in the past
        today = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
        if start_date_obj < today:
            response = {
                "success": False,
                "message": "Invalid date. Time off requests cannot be made for dates in the past."
            }
            return json.dumps(response)
        
        # Calculate the end date
        end_date_obj = start_date_obj + timedelta(days=number_of_days - 1)
        end_date = end_date_obj.strftime("%Y-%m-%d")
        
        # Return a confirmation request
        response = {
            "success": True,
            "requires_confirmation": True,
            "message": f"Please confirm your time off request for {start_date} to {end_date} ({number_of_days} days)",
            "request_details": {
                "start_date": start_date,
                "end_date": end_date,
                "number_of_days": number_of_days
            }
        }
        
        return json.dumps(response)
    except ValueError:
        response = {
            "success": False,
            "message": "Invalid date format. Please use YYYY-MM-DD format."
        }
        return json.dumps(response)
    except Exception as e:
        response = {
            "success": False,
            "message": f"Error processing request: {str(e)}"
        }
        return json.dumps(response)

#### 3.3.3 Human Decision Implementation

The `confirm_time_off_request()` tool implements the human's decision, either confirming or canceling the time off request:

In [ ]:
@tool
def confirm_time_off_request(start_date: str, end_date: str, number_of_days: int, confirm: bool) -> str:
    """Confirm or cancel a time off request.
    
    Args:
        start_date: The date that time off starts (format: YYYY-MM-DD)
        end_date: The date that time off ends (format: YYYY-MM-DD)
        number_of_days: The number of days requested off
        confirm: Boolean indicating whether to confirm (True) or cancel (False) the request
    
    Returns:
        A JSON string containing the result of the confirmation.
    """
    try:
        if not confirm:
            response = {
                "success": True,
                "message": "Time off request cancelled."
            }
            return json.dumps(response)
        
        # Create a new request
        request = {
            "start_date": start_date,
            "end_date": end_date,
            "number_of_days": number_of_days,
            "status": "pending"
        }
        
        # Add the request to the pending requests
        employee_time_off["pending_requests"].append(request)
        
        response = {
            "success": True,
            "message": f"Time off request confirmed and submitted for {start_date} to {end_date} ({number_of_days} days)",
            "request": request
        }
        
        return json.dumps(response)
    except Exception as e:
        response = {
            "success": False,
            "message": f"Error confirming request: {str(e)}"
        }
        return json.dumps(response)

### 3.4 Event Loop Tracking

The event loop tracker function provides transparency into the agent's operation

In [ ]:
def event_loop_tracker(**kwargs):
    # Track event loop lifecycle
    if kwargs.get("init_event_loop", False):
        print("🔄 Event loop initialized")
    elif kwargs.get("start_event_loop", False):
        print("▶️ Event loop cycle starting")
    elif kwargs.get("start", False):
        print("📝 New cycle started")
    elif "message" in kwargs:
        print(f"📬 New message created: {kwargs['message']['role']}")
    elif kwargs.get("complete", False):
        print("✅ Cycle completed")
    elif kwargs.get("force_stop", False):
        print(f"🛑 Event loop force-stopped: {kwargs.get('force_stop_reason', 'unknown reason')}")

    # Track tool usage
    if "current_tool_use" in kwargs and kwargs["current_tool_use"].get("name"):
        tool_name = kwargs["current_tool_use"]["name"]
        print(f"🔧 Using tool: {tool_name}")

### 3.5 Agent Creation and Configuration

The `create_hr_assistant()` function configures and creates the agent with the necessary tools:

In [ ]:
def create_hr_assistant():
    """Create and configure the HR assistant agent."""
    # Load configuration
    config_path = os.path.join(os.getcwd(), "BedrockAgentStack", "config.json")
    with open(config_path, "r") as config_file:
        config = json.load(config_file)
    
    # Get the Bedrock session
    bedrock_session = get_bedrock_session()
    
    # Create the Bedrock model with the client
    bedrock_model = BedrockModel(
        model_id=config["agentModelId"],
        # client=bedrock_client
        boto_session=bedrock_session
    )
    
    # Create the agent with system prompt, model, and tools
    agent = Agent(
        system_prompt=config["agentInstruction"],
        model=bedrock_model,
        tools=[get_time_off, request_time_off, confirm_time_off_request],
        callback_handler=event_loop_tracker
    )
    
    return agent

### 3.6 Main Function

The main function creates the agent and starts an interactive loop for user interaction:

In [ ]:

"""Main function to run the HR assistant agent."""
agent = create_hr_assistant()

print("HR Assistant Agent is ready!")
print("You can now interact with the agent.")

user_input = input("\nYou: I want to take 3 holidays around the 4th of July weekend")

# Process the user input and get a response
response = agent(user_input)

# Print the response
print(f"\nHR Assistant: {response}")

## 4. Best Practices for Human-in-the-Loop AI Systems
In this tutorial, we have demonstrated several best practices for implementing HITL in agentic AI systems:

#### **Explicit Confirmation for Consequential Actions** <br>

The time off request process in the tutorial follows a two-phase commit pattern:
1. **Request Phase**: The user initiates a time off request with start date and duration
2. **Confirmation Phase**: The user explicitly confirms or cancels the request
This pattern ensures that users have a chance to review and approve actions before they're finalized, reducing errors and increasing user confidence.

It is a good practice to always require explicit user confirmation before taking actions with significant consequences. 

#### **Clear Communication** <br>

Provide clear, concise information to users about:
- What the system is doing
- What information it needs from the user
- What the consequences of actions will be
- Any errors or issues that arise

#### **Appropriate Automation Level** <br>

Choose the appropriate level of automation for each task:
- Fully automate low-risk, routine tasks (like checking time off balance)
- Implement HITL for higher-risk or consequential actions (like submitting time off requests)

#### **Graceful Error Handling** <br>

Implement robust error handling with clear, actionable feedback to users. In our example, date format errors and past date errors are caught and explained clearly to the user.

#### **Transparent Operation** <br>

In this tutorial, the event loop tracker provides visibility into the agent's operations, printing messages about what the agent is doing at each step. This transparency helps users understand the system's behavior and builds trust.

Best pracitce: Make the system's operation transparent to users through appropriate logging, status updates, and explanations. 

#### **User-Centered Design** <br>

In this tutorial, the HR asisstant presents information in a progressive manner:
1. First showing available time off balance
2. Then presenting request details for confirmation
3. Finally confirming the submission

The system validates inputs (like date formats and whether dates are in the past) and provides clear, human-readable error messages. This helps users understand and correct issues without frustration.

Best practice: Design the HITL interactions around user needs and capabilities, not system architecture. Consider the user's mental model, cognitive load, and context when designing interaction patterns.

## 5. Conclusion

Human-in-the-Loop is a powerful paradigm for agentic AI systems that balances automation with human oversight and control. By implementing HITL patterns like confirmation requests, progressive disclosure, and transparent operation, we can create AI systems that are more trustworthy, effective, and aligned with human values and needs.

The HR assistant example demonstrates how these patterns can be implemented in practice, creating a system that automates routine tasks while keeping humans in control of consequential decisions. This approach combines the efficiency of automation with the judgment and oversight that only humans can provide.